In [ ]:
## Getting Started


### Install Vertex AI SDK for Python


! pip3 install --upgrade --user google-cloud-aiplatform

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, it is recommended to restart the runtime. Run the following cell to restart the current kernel.

The restart process might take a minute or so.


import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

After the restart is complete, continue to the next step.


<div class="alert alert-block alert-warning">
<b>⚠️ Wait for the kernel to finish restarting before you continue. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Define Google Cloud project information (Colab only)

If you are running this notebook on Google Colab, specify the Google Cloud project information to use. In the following cell, you specify your project information, import the Vertex AI package, and initialize the package. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = "learned-vehicle-408217"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries


from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)
import re

In [ ]:
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

prompt = """
Generate a factual list of the top 20 most visited countries in the world, ranked by the number of international tourist arrivals in the most recent year with available data.

Format:

List the countries in descending order of visitor numbers, starting with the most visited country.
Include the capital for each country and the main language spoken.
Present the information in a clear and concise bulleted list format.
Additional Guidance:

Strive for factual accuracy and precision in the generated list.
Prioritize clarity and readability in the output format.
If possible, cite the source of the data for transparency and credibility.
Explanation:

Clear Intent: The prompt explicitly states the desired output: a factual list of the most visited countries.
Specific Requirements: It outlines the desired format (bulleted list), ranking criteria (visitor numbers), and scope (top 20 countries).
Focus on Accuracy and Clarity: It emphasizes factual correctness and easy-to-understand presentation.
Optional Data Source Citation: It suggests citing data sources for added credibility.
"""

responses = model.generate_content(
    prompt,
    # "Enumarate the top 20 most visited cities in the world",
    generation_config=generation_config,
    stream=True,
)

res = []
countries = []
full_text = ""

for response in responses:
    full_text = full_text + response.text

countries = full_text.split('\n\n\n')

countries_list = []

for country in countries:
  if re.search("Capital", country):
    country_regex = r"\*\*(.*?)\*\*"
    nation = re.search(country_regex, country)

    country_name = nation.group(1) if nation else None

    capital = re.findall(r"Capital: (.+)", country)[0].strip()
    language = re.findall(r"Main language: (.+)", country)[0].strip()

    countries_list.append({
        "country": country_name,
        "capital": capital,
        "language": language,
    })

print(countries_list)

In [ ]:
# We are going to choose one of them. For example the first one
country = "United States"
capital = "New York"
language = "English"

print(country)

In [ ]:
prompt_names = """
Generate a factual list of the top 40 most used names in {} in the most recent year with available data.

Format:

List the names in descending order, starting with the most used name.
Present the information in a clear and concise bulleted list format.

Additional Guidance:

Strive for factual accuracy and precision in the generated list.
Prioritize clarity and readability in the output format.
If possible, cite the source of the data for transparency and credibility.

Explanation:

Clear Intent: The prompt explicitly states the desired output: a factual list of the most visited countries.
Specific Requirements: It outlines the desired format (bulleted list), ranking criteria (visitor numbers), and scope (top 20 countries).
Focus on Accuracy and Clarity: It emphasizes factual correctness and easy-to-understand presentation.
Optional Data Source Citation: It suggests citing data sources for added credibility.
""".format(country)

responses_names = model.generate_content(
    prompt_names,
    generation_config=generation_config,
    stream=True,
)

full_names_text = ""

for response in responses_names:
  # print(response.text)
  full_names_text = full_names_text + response.text


In [ ]:

names = re.findall(r"-\s*(.+)\n", full_names_text)

print(names)

In [ ]:
prompt_places = """
Generate a factual list of the top 40 most visited places in the city {} of {} in the most recent year with available data.

Format:

List the place names in descending order, starting with the most used name.
Present the information in a clear and concise bulleted list format.

Additional Guidance:

Strive for factual accuracy and precision in the generated list.
Prioritize clarity and readability in the output format.
If possible, cite the source of the data for transparency and credibility.

Explanation:

Clear Intent: The prompt explicitly states the desired output: a factual list of the most visited places.
Specific Requirements: It outlines the desired format (bulleted list), ranking criteria (visitor numbers), and scope (top 40 places).
Focus on Accuracy and Clarity: It emphasizes factual correctness and easy-to-understand presentation.
Optional Data Source Citation: It suggests citing data sources for added credibility.
""".format(capital, country)

responses_places = model.generate_content(
    prompt_places,
    generation_config=generation_config,
    stream=True,
)

full_names_places = ""

for response in responses_places:
  print(response.text)
  full_names_places = full_names_places + response.text

In [ ]:
places = re.findall(r".\t\s*(.+)\n", full_names_places)

print(places)

In [ ]:
prompt_events = """
Generate a factual list of the top 40 most multicultural events in the city {} of {}.

Format:

List the events in descending order of visitor numbers, starting with the most visited country.
Include the location for each event and the event category.
Present the information in a clear and concise bulleted list format.
Additional Guidance:

Strive for factual accuracy and precision in the generated list.
Prioritize clarity and readability in the output format.
If possible, cite the source of the data for transparency and credibility.
Explanation:

Clear Intent: The prompt explicitly states the desired output: a factual list of the most visited events.
Specific Requirements: It outlines the desired format (bulleted list), and scope (top 40 event).
Focus on Accuracy and Clarity: It emphasizes factual correctness and easy-to-understand presentation.
Optional Data Source Citation: It suggests citing data sources for added credibility.
""".format(capital, country)

responses_events = model.generate_content(
    prompt_events,
    generation_config=generation_config,
    stream=True,
)

full_names_events = ""

for response in responses_events:
  print(response.text)
  full_names_events = full_names_events + response.text


In [ ]:
prompt_events = """
Generate a factual list of 40 multicultural events in the city {} of {}.

Format:

List the events in descending alphabet order.
Include the location for each event and the event category.
Present the information in a clear and concise bulleted list format.
Additional Guidance:

Strive for factual accuracy and precision in the generated list.
Prioritize clarity and readability in the output format.
If possible, cite the source of the data for transparency and credibility.
Explanation:

Clear Intent: The prompt explicitly states the desired output: a factual list of the events.
Specific Requirements: It outlines the desired format (bulleted list), and scope (top 40 event).
Focus on Accuracy and Clarity: It emphasizes factual correctness and easy-to-understand presentation.
Optional Data Source Citation: It suggests citing data sources for added credibility.
""".format(capital, country)

responses_events = model.generate_content(
    prompt_events,
    generation_config=generation_config,
    stream=True,
)

full_names_events = ""

for response in responses_events:
  print(response.text)
  full_names_events = full_names_events + response.text
